In [1]:
!pip install "vegafusion[embed]>=1.5.0" "vegafusion>=1.5.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 34.7 MB/s eta 0:00:00


In [2]:
!pip install "vl-convert-python>=1.6.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 15.6 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Impor Library
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import altair as alt

In [5]:
# Load dataset
df = pd.read_csv('https://drive.google.com/uc?export=download&id=1JsO7arVimyOXf2R8rTVi2LCt76PX306n')
df

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6870,139.7495,Japan,JP,JPN,Tōkyō,primary,37785000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629
...,...,...,...,...,...,...,...,...,...,...,...
48054,Al Jabīn,Al Jabin,14.7040,43.5990,Yemen,YE,YEM,Raymah,admin,NaN,1887910100
48055,Nelspruit,Nelspruit,-25.4745,30.9703,South Africa,ZA,ZAF,Mpumalanga,admin,NaN,1710114438
48056,Gqeberha,Gqeberha,-33.9681,25.5981,South Africa,ZA,ZAF,Eastern Cape,NaN,NaN,1710000082
48057,Lupane,Lupane,-18.9315,27.8070,Zimbabwe,ZW,ZWE,Matabeleland North,admin,NaN,1716206606


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48059 entries, 0 to 48058
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        48059 non-null  object 
 1   city_ascii  48057 non-null  object 
 2   lat         48059 non-null  float64
 3   lng         48059 non-null  float64
 4   country     48059 non-null  object 
 5   iso2        48026 non-null  object 
 6   iso3        48059 non-null  object 
 7   admin_name  47858 non-null  object 
 8   capital     15138 non-null  object 
 9   population  47808 non-null  float64
 10  id          48059 non-null  int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 4.0+ MB


In [7]:
df.duplicated().sum()

np.int64(0)

In [8]:
df.isna().sum()

,0
city,0
city_ascii,2
lat,0
lng,0
country,0
iso2,33
iso3,0
admin_name,201
capital,32921
population,251


In [9]:
# Seleksi fitur
coords = df[['lat', 'lng']].values

In [10]:
# Konstanta Radius rata-rata Bumi dalam km
kms_per_radian = 6371.0088

# Konversi koordinat dari Derajat ke Radian
coords_rad = np.radians(coords)

# Tentukan parameter DBSCAN
eps_in_km = 20         # Radius Epsilon: 20 km
min_samples = 5        # MinPts: minimal 5 kota
eps_in_radians = eps_in_km / kms_per_radian   # Konversi eps ke radian

In [11]:
# Membangun Model DBSCAN
db = DBSCAN(
    eps=eps_in_radians,
    min_samples=min_samples,
    metric='haversine'
).fit(coords_rad)

# Tambahkan hasil label cluster ke DataFrame
labels = db.labels_
df['cluster_label'] = labels

# Simpan DataFrame yang kini memiliki kolom 'cluster_label' ke CSV
df.to_csv('worldcities_clustered_final.csv', index=False)

# Analisis Hasil
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = list(labels).count(-1)

print(f"Jumlah Cluster Ditemukan  : {n_clusters}")
print(f"Jumlah Titik Noise (-1)   : {n_noise} (Persentase: {n_noise / len(df) * 100:.2f}%)")

Jumlah Cluster Ditemukan  : 926
Jumlah Titik Noise (-1)   : 21717 (Persentase: 45.19%)


In [12]:
# Muat kembali data yang sudah memiliki kolom 'cluster_label'
df_clustered = pd.read_csv('worldcities_clustered_final.csv')

# Pisahkan Noise (-1) dan Cluster untuk visualisasi warna
df_noise = df_clustered[df_clustered['cluster_label'] == -1]

# Ambil sampel 20% dari cluster (jika datanya terlalu besar untuk Colab)
df_clusters = df_clustered[df_clustered['cluster_label'] != -1].sample(frac=0.2, random_state=42)
df_plot = pd.concat([df_noise, df_clusters])

# Tambahkan kolom penentu warna
df_plot['color_type'] = df_plot['cluster_label'].apply(lambda x: 'Noise' if x == -1 else 'Cluster')

# Aktifkan VegaFusion agar Altair bisa mengatasi data besar tanpa error
alt.data_transformers.enable("vegafusion")

# Membuat peta interaktif
chart = alt.Chart(df_plot).mark_circle(size=20).encode(
    longitude='lng:Q',
    latitude='lat:Q',
    tooltip=['city', 'country', 'cluster_label', 'population'],
    # Titik Merah = Noise, Titik Biru = Cluster
    color=alt.Color('color_type:N',
                    scale=alt.Scale(domain=['Noise', 'Cluster'], range=['red', 'blue']),
                    title='Keterangan'),
    opacity=alt.value(0.7)
).properties(
    title='Visualisasi DBSCAN: Cluster (Biru) vs Noise (Merah)'
).interactive()

# Simpan hasil visualisasi
chart.save('dbscan_worldcities_map_final.json')

# Simpan data hasil clustering
df.to_csv('worldcities_clustered_final.csv', index=False)

# Tampilkan Peta
chart.display()

Output hidden; open in https://colab.research.google.com to view.

In [13]:
# Muat kembali data hasil clustering
df_clustered = pd.read_csv('worldcities_clustered_final.csv')

# Mengambil data yang termasuk label cluster (>= 0)
clustered_data = df_clustered[df_clustered['cluster_label'] != -1].copy()

# Ambil koordinat dan label dari data yang sudah difilter
coords_filtered = clustered_data[['lat', 'lng']].values
labels_filtered = clustered_data['cluster_label'].values

# Konversi Koordinat yang sudah difilter ke satuan Radian
coords_filtered_rad = np.radians(coords_filtered)

# Hitung Skor Silhouette
try:
    silhouette_avg = silhouette_score(
        coords_filtered_rad,
        labels_filtered,
        metric='haversine' # Menggunakan metrik yang sama dengan DBSCAN
    )

    print("\n===== HASIL SKOR SILHOUETTE =====")
    print(f"Rata-rata Skor Silhouette : {silhouette_avg:.4f}")

    # Interpretasi hasil
    if silhouette_avg > 0.5:
        print("Interpretasi : Klaster terbentuk dengan sangat baik (kualitas tinggi).")
    elif silhouette_avg > 0.2:
        print("Interpretasi : Klaster terbentuk dengan wajar, ada sedikit tumpang tindih.")
    else:
        print("Interpretasi : Kualitas klaster rendah, mungkin perlu penyesuaian parameter.")

except ValueError as e:
    print(f"\nERROR saat menghitung Skor Silhouette : {e}")


===== HASIL SKOR SILHOUETTE =====
Rata-rata Skor Silhouette : 0.2282
Interpretasi : Klaster terbentuk dengan wajar, ada sedikit tumpang tindih.
